# Materials and Emission Streams

This is a continuation of the Multiscale MILP example

We will be adding the following considerations: 

1. material for the establishment of operations 
2. global warming potential induced as a sum of our decisions and choices 

## See Multiscale MILP

In [1]:
from energia import *

m = Model('example3')
m.q = Periods()
m.y = 4 * m.q

In [2]:
m.usd = Currency()
m.declare(Resource, ['power', 'wind', 'solar'])
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49

m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10
m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

m.lii = Storage()
m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432
m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.charge.operate <= 1
m.lii.discharge.capacity <= 100
m.lii.discharge.operate <= 1

⚖   Initiated solar balance in (l0, q)                                      ⏱ 0.0002 s
🔗  Bound [≤] solar consume in (l0, q)                                       ⏱ 0.0012 s
⚖   Initiated wind balance in (l0, y)                                       ⏱ 0.0001 s
🔗  Bound [≤] wind consume in (l0, y)                                        ⏱ 0.0006 s
⚖   Initiated power balance in (l0, q)                                      ⏱ 0.0001 s
🔗  Bound [≥] power release in (l0, q)                                       ⏱ 0.0010 s
🔗  Bound [≤] wf capacity in (l0, y)                                         ⏱ 0.0002 s
🔗  Bound [≥] wf capacity in (l0, y)                                         ⏱ 0.0001 s
🔗  Bound [≤] wf operate in (l0, q)                                          ⏱ 0.0003 s
🧭  Mapped time for operate (wf, l0, q) ⟺ (wf, l0, y)                        ⏱ 0.0001 s
🔗  Bound [≤] pv capacity in (l0, y)                                         ⏱ 0.0002 s
🔗  Bound [≥] pv capacity in (l0, y)        

## Indicator Stream 

Indicators scale the impact of some decision or flow and project it only a common dimension allowing the calculation of overall impact as a single metric value

In this case, we consider global warming potential (GWP).

In [3]:
m.gwp = Environ()

Now say that we want to consider the impact of setting up operations (specific to the act of construction)

These can be provided as calculations 

In [4]:
m.wf.capacity[m.gwp.emit] == 1000
m.pv.capacity[m.gwp.emit] == 2000
m.lii.capacity[m.gwp.emit] == 3000
m.emit.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Note that impact streams are mapped across scales but not balanced, unless a negative (relatively) impact is also assessed! In the case of emission, this would be abatement.

## Material Stream

Note that all the objects found in energia.components.commodity.misc are still resources (Land, Material, etc.)

This will allow for the modeling of an expansive type of processes.

Examples:

1. land clearance to transform agricultural land into industrial, or land remediation 
2. resource intense material recycling 
3. some resource flowing being treated as emission flows 

The possibilities are vast... The advanced modeler may prefer to use Resource() for everything [see Resource Task Network (RTN) Framework]

In the following example, we consider a limit to cement consumption, as also an expense and gwp impact associated with it

In [5]:
m.cement = Material()
m.cement.consume <= 1000000
m.cement.consume[m.usd.spend] == 17
m.cement.consume[m.gwp.emit] == 0.9

⚖   Initiated cement balance in (l0, y)                                     ⏱ 0.0001 s
🔗  Bound [≤] cement consume in (l0, y)                                      ⏱ 0.0010 s


Next, we provide details of the use of cement across all operations 

In [6]:
m.wf.capacity[m.cement.use] == 400
m.pv.capacity[m.cement.use] == 560
m.lii.capacity[m.cement.use] == 300

⚖   Updated cement balance with use(cement, l0, y, capacity, wf)            ⏱ 0.0001 s
⚖   Updated cement balance with use(cement, l0, y, capacity, pv)            ⏱ 0.0001 s
⚖   Updated cement balance with use(cement, l0, y, invcapacity, lii.stored) ⏱ 0.0001 s


Material use is summed up across all operations, and adheres to an upper bound in terms of consumption. 

In [7]:
m.cement.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

GWP impact is now the sum of impact from material use as well as constuction 

In [8]:
m.emit.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Locate Operations

In [9]:
m.network.locate(m.wf, m.pv, m.lii)

⚖   Updated power balance with produce(power, l0, q, operate, wf)           ⏱ 0.0001 s
⚖   Updated wind balance with expend(wind, l0, y, operate, wf)              ⏱ 0.0001 s
🏭  Operating streams introduced for wf in l0                                ⏱ 0.0022 s
🏗  Construction streams introduced for wf in l0                             ⏱ 0.0000 s
🌍  Located wf in l0                                                         ⏱ 0.0029 s
⚖   Updated power balance with produce(power, l0, q, operate, pv)           ⏱ 0.0001 s
⚖   Updated solar balance with expend(solar, l0, q, operate, pv)            ⏱ 0.0001 s
🏭  Operating streams introduced for pv in l0                                ⏱ 0.0020 s
🏗  Construction streams introduced for pv in l0                             ⏱ 0.0000 s
🌍  Located pv in l0                                                         ⏱ 0.0028 s
💡  Assumed lii capacity unbounded in (l0, y)                                ⏱ 0.0001 s
🧭  Mapped time for inventory (lii.stored, l

## Optimization 

Let us minimize GWP this time. Note that m.usd.spend.opt() can still be used!

In [10]:
m.gwp.emit.opt()

🧭  Mapped samples for emit (gwp, l0, y, capacity, wf) ⟺ (gwp, l0, y)        ⏱ 0.0001 s
🧭  Mapped samples for emit (gwp, l0, y, capacity, pv) ⟺ (gwp, l0, y)        ⏱ 0.0002 s
🧭  Mapped samples for emit (gwp, l0, y, invcapacity, lii.stored) ⟺ (gwp, l0, y) ⏱ 0.0002 s
🧭  Mapped samples for emit (gwp, l0, y, consume, cement) ⟺ (gwp, l0, y)     ⏱ 0.0002 s
📝  Generated Program(example3).mps                                          ⏱ 0.0038 s


Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-01
Read MPS format model from file Program(example3).mps
Reading time = 0.00 seconds
PROGRAM(EXAMPLE3): 95 rows, 87 columns, 217 nonzeros


📝  Generated gurobipy model. See .formulation                               ⏱ 0.0079 s


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 95 rows, 87 columns and 217 nonzeros
Model fingerprint: 0x39a95d48
Variable types: 84 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 1e+06]
Presolve removed 83 rows and 76 columns
Presolve time: 0.00s
Presolved: 12 rows, 11 columns, 38 nonzeros
Variable types: 11 continuous, 0 integer (0 binary)

Root relaxation: objective 4.752148e+05, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    475214.81481 475214.815  0.0

📝  Generated Solution object for Program(example3). See .solution           ⏱ 0.0004 s
✅  Program(example3) optimized using gurobi. Display using .output()        ⏱ 0.0185 s


In [11]:
len(m.variables)

87

In [12]:
m.show()

# Mathematical Program for Program(example3)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###  Function Sets

<IPython.core.display.Math object>

In [13]:
m.output(False)

# Solution for Program(example3)

<br><br>

## Objective

<IPython.core.display.Math object>

<br><br>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>